Find large MBAs suitable for MRS fringe characterization.  For now: restrict to objects with numbers between 200 and 800 (chosen arbitrarily).  May contain, as false positives, near-Earth asteroids that happen to be in the main belt at the time. <br />
This notebook modified after 'centaurs2019.ipynb' used for GTO planning.

M.Mueller@astro.rug.nl, 2021/09/27

Find large MBAs observable with JWST during early routine science. <br />
Step 1: get ephemerides for objects 200--800, store objects that are in the JWST field of regard for some time.  Filter for heliocentric distance (must be within asteroid belt). <br />
Step 2: estimate (reflected + thermal) flux, compare against sensitivity.  Output which objects are doable at what wavelength.  <br />
<br />
Caveat 1: not checking for ephemeris uncertainty at this point -- probably not an issue for these well-characterized objects. <br />
Caveat 2: after repeated runs, Horizons may block your URL for some time due to an excessive number of API calls.  Do we need to get whitelisted?  Is it a good idea to pre-calculate ephemerides rather than requesting them over and over again?

In [1]:
from __future__ import print_function
import neatm
import astropy.units as u
from astropy.table import Table
from scipy import interpolate
from urllib.request import urlopen
import numpy as np
import os
from astroquery.jplhorizons import Horizons

## Define global parameters

In [2]:
rMin=2 # in AU
rMax = 4.5 # 
minGlxLat = 5 # in degree  ### not implemented for now
minNumber=300 # the very brightest asteroids might saturate
maxNumber=1000
maxNumber=350 # for demo purposes; shorter run-time

In [3]:
startDate = '2022-07-01'
endDate   = '2022-07-11'
center='@JWST'
elongLimits = [85,135]

In [4]:
eta=0.8
pv=0.2 # Somewhat conservative.  Good assumption for S types, conservative for C types, which are the most abundant types

In [5]:
# fluxes to get SNR~1,000 in ~1,000s on-source time per dichroic at native spectral resolution
# See ETC notebook 89119 (which needs some attention, still)

minFlux = {}  
## The following fluxes give SNR = 300 in ~1.000s on-source time (per dichroic) at native spectral resolution.
## Taken from ETC: 2 dithers times 2 exposures (to mimic 4-point dither) times 100 FAST frames, 1,110 sec; high bg
minFlux[10*u.micron]=50*u.mJy 
minFlux[20*u.micron]=250*u.mJy 
wavelengths = [key for key in minFlux]

In [6]:
# epochs needed by astroquery.jplhorizons.Horizons
epochs={}
epochs['start']=startDate
epochs['stop']=endDate
epochs['step']='1d' # hard-wire: daily output

## Aux function

In [7]:
sunFluxFile='Rieke2008.fluxSunVega.txt'
assert os.path.isfile(sunFluxFile)
# Modified after Lenka's "Ref_Brightness.py" as of 2018/03/20: https://github.com/lenkaaaa49/Serendipitous-asteroid-observation
solar_flux_density=Table.read('Rieke2008.fluxSunVega.txt', guess=False,format='ascii.fixed_width',  delimiter=' ',
                       header_start=None, data_start=14,
                       col_starts=(0, 8, 17),
                      col_ends=(7, 16, 26),names=('Wavelength[micron]','Flux density of the Sun[W/m2/nm]',
                            'Flux density of Vega[W/m2/nm]'))
lambdasSun=solar_flux_density['Wavelength[micron]'].data
fluxSun = solar_flux_density['Flux density of the Sun[W/m2/nm]'].data
s = interpolate.InterpolatedUnivariateSpline(lambdasSun, fluxSun)
solarFluxes = np.array([(s(wav.to_value(u.micron))*u.Watt/u.meter**2/u.nm).to_value(u.mJy, equivalencies=u.spectral_density(wav)) for wav in wavelengths])*u.mJy

vSun=-26.74

def reflectedSunlight(v,relRefl=1.4): 
    # assume wavelengths defined globally....
    return solarFluxes*10**(-(v-vSun)/2.5)*relRefl

## Retrieve ephemerides, filter

In [8]:
fringeAsteroids={}
for i in range(minNumber,maxNumber+1):
    # get ephemeris
    hor=Horizons( id='%s'%i, location=center, epochs=epochs )
    try:
        asteroid = hor.ephemerides(solar_elongation=elongLimits) 
    except ValueError:
        print( i, ", not in field of regard")
        continue
        
    observable = np.where( (asteroid['r']<rMax) & (asteroid['r']>rMin) )
    if len(observable[0])>0:
        print (i)
        fringeAsteroids[i]=asteroid[observable]
    else:
        print ('.')

300
301 , not in field of regard
302 , not in field of regard
303 , not in field of regard
304
305
306
307 , not in field of regard
308 , not in field of regard
309
310 , not in field of regard
311 , not in field of regard
312 , not in field of regard
313 , not in field of regard
314 , not in field of regard
.
316 , not in field of regard
317 , not in field of regard
318 , not in field of regard
319
320 , not in field of regard
321 , not in field of regard
322 , not in field of regard
323
324 , not in field of regard
325 , not in field of regard
326 , not in field of regard
327
328 , not in field of regard
329
330 , not in field of regard
331 , not in field of regard
332
333 , not in field of regard
334 , not in field of regard
335 , not in field of regard
336 , not in field of regard
337 , not in field of regard
338 , not in field of regard
339 , not in field of regard
340
.
342
343 , not in field of regard
344 , not in field of regard
345
346 , not in field of regard
347
348 , not in

In [9]:
with open('fringeAsteroids.txt', 'w') as f:
    for obj in fringeAsteroids:
        eph = fringeAsteroids[obj]
        print (obj)
        f.write(str(obj)+ '\n')
        f.write('# H=%s\n'%eph['H'][0])
        f.write('# G=%s\n'%eph['G'][0])
        for t, v, r, delta,alpha in zip(eph['datetime_jd'], eph['V'], eph['r'], eph['delta'], eph['alpha']):
            f.write('%.1f, %.2f, %.5f, %.5f, %.2f\n'%(t,v,r,delta,alpha))
        f.write('\n')

300
304
305
306
309
319
323
327
329
332
340
342
345
347


## Estimate fluxes, compare against sensitivity

In [10]:
brightEnough={}
print( "During the selected epoch, asteroid number N is observable for M days with sufficient brightness:")
print( "" )
for obj in fringeAsteroids.keys():
    brightEnough[obj]={}
    for wav in wavelengths:
        brightEnough[obj][wav]=[] # tuples: (datetime,flux)
    eph=fringeAsteroids[obj]
    h=eph['H'][0]
    g=eph['G'][0]
    for t, v, r, delta,alpha in zip(eph['datetime_str'], eph['V'], eph['r'], eph['delta'], eph['alpha']):
        thermal=neatm.neatm(h,g, alpha*u.deg, r*u.AU, delta*u.AU, wavelengths, eta, pv, mJy=True)
        thermal=np.array([x.to_value(u.mJy) for x in thermal])*u.mJy
        reflected=reflectedSunlight(v)
        total=reflected+thermal
        for l,f in zip(wavelengths, total):
            if f.to_value(u.mJy) > minFlux[l].to_value(u.mJy):
                brightEnough[obj][l].append((t,f))
    print (obj)
    for wav in wavelengths:
        print(wav, len(brightEnough[obj][wav]))
    print()


During the selected epoch, asteroid number N is observable for M days with sufficient brightness:

300
10.0 micron 11
20.0 micron 11

304
10.0 micron 11
20.0 micron 11

305
10.0 micron 5
20.0 micron 5

306
10.0 micron 11
20.0 micron 11

309
10.0 micron 2
20.0 micron 2

319
10.0 micron 11
20.0 micron 11

323
10.0 micron 11
20.0 micron 11

327
10.0 micron 11
20.0 micron 11

329
10.0 micron 11
20.0 micron 11

332
10.0 micron 11
20.0 micron 11

340
10.0 micron 8
20.0 micron 8

342
10.0 micron 11
20.0 micron 11

345
10.0 micron 8
20.0 micron 8

347
10.0 micron 11
20.0 micron 11

